In [3]:
import statistics
import math
import scipy.stats
from IPython.display import Latex

# Input Choices

In [2]:
dimming_disabled = [
    {"attrition": 2327, "items_checked_out": 14, "recommendations_checked_out": 7},
    {"attrition": 2205, "items_checked_out": 84, "recommendations_checked_out": 44},
    {"attrition": 2355, "items_checked_out": 17, "recommendations_checked_out": 14},
    {"attrition": 2343, "items_checked_out": 12, "recommendations_checked_out": 9},
    {"attrition": 2228, "items_checked_out": 70, "recommendations_checked_out": 42},
]

baseline_dimming = [
    {"attrition": 1173, "items_checked_out": 2393, "recommendations_checked_out": 884},
    {"attrition": 1228, "items_checked_out": 2387, "recommendations_checked_out": 882},
    {"attrition": 1233, "items_checked_out": 2301, "recommendations_checked_out": 871},
    {"attrition": 1237, "items_checked_out": 2298, "recommendations_checked_out": 862},
    {"attrition": 1290, "items_checked_out": 2188, "recommendations_checked_out": 938},
]

component_weightings = [
    {"attrition": 1027, "items_checked_out": 2688, "recommendations_checked_out": 1412},
    {"attrition": 992, "items_checked_out": 2645, "recommendations_checked_out": 1432},
    {"attrition": 989, "items_checked_out": 2722, "recommendations_checked_out": 1437},
    {"attrition": 1014, "items_checked_out": 2726, "recommendations_checked_out": 1469},
    {"attrition": 982, "items_checked_out": 2711, "recommendations_checked_out": 1445},
]

profiling_without_component_weightings = [
    {"attrition": 1170, "items_checked_out": 2181, "recommendations_checked_out": 1091},
    {"attrition": 1238, "items_checked_out": 2042, "recommendations_checked_out": 1022},
    {"attrition": 1195, "items_checked_out": 2187, "recommendations_checked_out": 1121},
    {"attrition": 1281, "items_checked_out": 1986, "recommendations_checked_out": 993},
    {"attrition": 1209, "items_checked_out": 2051, "recommendations_checked_out": 1053},
]

profiling_with_component_weightings = [
    {"attrition": 1169, "items_checked_out": 2192, "recommendations_checked_out": 1175},
    {"attrition": 1211, "items_checked_out": 2068, "recommendations_checked_out": 1100},
    {"attrition": 1221, "items_checked_out": 2128, "recommendations_checked_out": 1131},
    {"attrition": 1188, "items_checked_out": 2192, "recommendations_checked_out": 1174},
    {"attrition": 1194, "items_checked_out": 2142, "recommendations_checked_out": 1152},
]

# Experiment Outputs

## LaTeX table

In [12]:
source = profiling_with_component_weightings

###################################################


def avg(runs, key):
    return sum([run[key] for run in runs]) / len(runs)


def stdev(runs, key):
    return statistics.stdev([run[key] for run in runs])


def latex_table(runs):
    preamble = (
        "\\begin{table}[ht]\n"
        "\\centering\n"
        "\\ra{1.3}\n"
        "\\begin{tabular}{@{}rlll@{}}\\toprule\n"
        "Run & \multicolumn{2}{l}{Items Checked Out} & Attrition \\\ \\cline{2-3}\n"
        "\\phantom{} & Total & of which Recommended &  \\\ \\midrule\n"
    )
    run_rows = [
        f"{i+1} & {run['items_checked_out']} & {run['recommendations_checked_out']} & {run['attrition']} \\\ \n"
        for i, run in enumerate(runs)
    ]
    avg_row = f"\midrule $avg$ & {round(avg(runs, 'items_checked_out'), 1)} & {round(avg(runs, 'recommendations_checked_out'), 1)} & {round(avg(runs, 'attrition'), 1)} \\\ \n"
    stdev_row = f"$s$ & {round(stdev(runs, 'items_checked_out'), 1)} & {round(stdev(runs, 'recommendations_checked_out'), 1)} & {round(stdev(runs, 'attrition'), 1)} \\\ \n"
    postamble = (
        "\\bottomrule\n"
        "\\end{tabular}\n"
        "\\caption{Shopping metrics for five constant load scenario runs with TODO.}\n"
        "\\label{fig:constant_load_TODO_repeated}\n"
        "\\end{table}\n"
    )
    return preamble + "".join(run_rows) + avg_row + stdev_row + postamble


print(latex_table(source))

\begin{table}[ht]
\centering
\ra{1.3}
\begin{tabular}{@{}rlll@{}}\toprule
Run & \multicolumn{2}{l}{Items Checked Out} & Attrition \\ \cline{2-3}
\phantom{} & Total & of which Recommended &  \\ \midrule
1 & 2192 & 1175 & 1169 \\ 
2 & 2068 & 1100 & 1211 \\ 
3 & 2128 & 1131 & 1221 \\ 
4 & 2192 & 1174 & 1188 \\ 
5 & 2142 & 1152 & 1194 \\ 
\midrule $avg$ & 2144.4 & 1146.4 & 1196.6 \\ 
$s$ & 51.6 & 31.6 & 20.3 \\ 
\bottomrule
\end{tabular}
\caption{Shopping metrics for five constant load scenario runs with TODO.}
\label{fig:constant_load_TODO_repeated}
\end{table}



## Hypothesis Testing

In [16]:
candidate = profiling_without_component_weightings
control = profiling_with_component_weightings
key = "items_checked_out"
# key = "recommendations_checked_out"
# key = "attrition"

###################################################


def welch_dof(x1, x2):
    var_x1 = statistics.variance(x1)
    var_x2 = statistics.variance(x2)
    len_x1 = len(x1)
    len_x2 = len(x2)
    numerator = (var_x1 / len_x1 + var_x2 / len_x2) ** 2
    denominator = (var_x1 ** 2) / ((len_x1 ** 2) * (len_x1 - 1)) + (var_x2 ** 2) / (
        (len_x2 ** 2) * (len_x2 - 1)
    )
    return numerator / denominator


def t(x1, x2):
    mean_x1 = sum(x1) / len(x1)
    mean_x2 = sum(x2) / len(x2)
    var_x1 = statistics.variance(x1)
    var_x2 = statistics.variance(x2)
    N_x1 = len(x1)
    N_x2 = len(x2)

    return (mean_x1 - mean_x2) / math.sqrt(var_x1 / N_x1 + var_x2 / N_x2)


dof = welch_dof([run[key] for run in control], [run[key] for run in candidate])
t_statistic = t([run[key] for run in control], [run[key] for run in candidate])

print(f"Degrees of freedom: {math.floor(dof)} (rounded down from {dof})")
print(f"Test statistic: {t_statistic}")
print(
    f"Critical values:"
    f"\n\t90th: {scipy.stats.t.ppf(q=0.90,df=math.floor(dof))}"
    f"\n\t95th: {scipy.stats.t.ppf(q=0.95,df=math.floor(dof))}"
    f"\n\t97.5th: {scipy.stats.t.ppf(q=0.975,df=math.floor(dof))}"
    f"\n\t99th: {scipy.stats.t.ppf(q=0.99,df=math.floor(dof))}"
    f"\n\t99.5th: {scipy.stats.t.ppf(q=0.995,df=math.floor(dof))}"
    f"\n\t99.9th: {scipy.stats.t.ppf(q=0.999,df=math.floor(dof))}\n"
)

print(f"Significant if {abs(t_statistic)} is greater than the chosen critical value if one-tailed")

Degrees of freedom: 6 (rounded down from 6.376172936645847)
Test statistic: 1.186541363357757
Critical values:
	90th: 1.4397557472577693
	95th: 1.9431802803927816
	97.5th: 2.4469118487916806
	99th: 3.142668403290985
	99.5th: 3.707428021324907
	99.9th: 5.207626238838042

Significant if 1.186541363357757 is greater than the chosen critical value if one-tailed
